# Langchain Prompt

---
## 1. Using PromptTemplate
- This is the most common and flexible method. You define a prompt with placeholders that get filled in dynamically.

In [37]:
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

output_parser = StrOutputParser()
# llm = ChatGroq(model="llama3-8b-8192")   # 8b model  - faster model
llm = ChatGroq(model="llama3-70b-8192")  # 70b model  x8 better but slower

#### Use PrompTemplate.from_template to pass in a single simple prompt text

In [ ]:
prompt = PromptTemplate.from_template("Write a funny 3 line poem about {topic}")
formatted_prompt = prompt.format(topic="cats")
result = llm.invoke(formatted_prompt)
print(result.content)

#### User PrompTemplate initialization with input_variables and template parametr and a LCEL

In [41]:
prompt = PromptTemplate(
    input_variables = ["product"],
    template = "List 3 benefits of using {product}"
)

chain = prompt | llm | output_parser

result = chain.invoke({"product":"soap"})
result

'Here are three benefits of using soap:\n\n1. **Cleansing**: Soap helps to remove dirt, oil, and other impurities from the skin, leaving it feeling clean and fresh. This is especially important for personal hygiene and preventing the spread of illnesses.\n2. **Skin Health**: Soap can help to maintain healthy skin by removing bacteria and other microorganisms that can cause infections and skin conditions like acne. Many soaps also contain moisturizing ingredients that help to keep the skin hydrated and soft.\n3. **Prevention of Infections**: Soap has been shown to be effective in preventing the spread of infections, including those caused by germs like influenza, norovirus, and MRSA. By washing your hands regularly with soap, you can reduce the risk of getting sick and prevent the spread of illnesses to others.'

####  you can also do

In [43]:
prompt = PromptTemplate(
    input_variables = ["product"],
    template = "List 3 benefits of using {product}"
)
final_prompt =prompt.format(product="soap")

chain = prompt | llm | output_parser
result = chain.invoke(final_prompt)
result

'Here are 3 benefits of using soap:\n\n1. **Cleans and removes dirt and germs**: Soap helps to remove dirt, grime, and germs from the skin, which can help to prevent the spread of illnesses and infections. By washing with soap, you can remove bacteria, viruses, and other microorganisms that can cause harm.\n2. **Moisturizes and softens skin**: Many soaps contain moisturizing ingredients that help to soften and hydrate the skin. This can be especially beneficial for people with dry or sensitive skin, as soap can help to lock in moisture and reduce irritation.\n3. **Freshens and deodorizes**: Soap can leave your skin feeling fresh and clean, and can also help to eliminate body odor. Many soaps contain fragrances or antibacterial agents that can help to reduce sweat and odor, leaving you feeling confident and fresh throughout the day.'

#### You can use Prompt_template.from_template to pass in a simple template

---
---
## 2. Using ChatPromptTemplate for Chat Models
- Designed for chat-based models (like OpenAI's gpt-3.5/4). Supports message roles (user, system, assistant).

In [18]:
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
# from langchain_core.messages import HumanMessage, SystemMessage
# llm and output_parser defined in previous cells

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are an AI assistant that only output 3 line funny poem"),
    HumanMessagePromptTemplate.from_template("write about {topic}")
])

chain  = prompt | llm | output_parser

result = chain.invoke({"topic":"third world war"})
print(result)

World War Three's a blast,
Nukes are flying fast,
We're all toast at last!


#### for ChatPromptTemplate you can also do

In [20]:
# ChatPromptTemplate takin a list of tuples options "system", "human","ai"
prompt = ChatPromptTemplate.from_messages([
   ("system","You are an AI assistant that only output 3 line funny poem"),
    ("human","write about {topic}")
])

chain  = prompt | llm | output_parser

formatted_prompt = prompt.format_messages(topic="artificial intelligence") ## pass in a format  instead of chain invoke
result = chain.invoke(formatted_prompt)
print(result)

AI's on the rise, they say,
Taking over soon, in a major way,
But first, it needs a coffee break!


---
---
## 3. FewShotPromptTemplate
- Creates prompts with multiple examples for few-shot learning.

In [26]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
]

prompt = PromptTemplate(
    input_variables = ["input","output"],
    template= "Input {input}\nOutput: {output}"
)

shot_prompt = FewShotPromptTemplate(
    examples= examples,
    example_prompt=prompt,
    prefix="Give the opposite of each word",
    suffix="Input {adjective}\nOutput:",
    input_variables=["adjective"]
)

chain = shot_prompt | llm | output_parser
result = chain.invoke({"adjective":"boy"})
print(result)

girl


In [35]:
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.prompts.chat import ChatPromptTemplate

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# full_prompt = ChatPromptTemplate.from_messages(
#     ("syste,","you are an Maths assistant"),
#     few_shot_prompt,
#     ("human","{input}")
# )

chain = few_shot_prompt | llm  #| output_parser
result = chain.invoke({"input":"5 + 7"})
print("Result is",result)

Result is content='' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 28, 'total_tokens': 29, 'completion_time': 0.00796435, 'prompt_time': 0.000491702, 'queue_time': -9223372036.855268, 'total_time': 0.008456052}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None} id='run--1fa92f7c-7e4c-4334-b35e-0002c1986c07-0' usage_metadata={'input_tokens': 28, 'output_tokens': 1, 'total_tokens': 29}


---
---
4. ## Importing prompt from langchain hub

In [72]:
from langchain import hub
 

# prompt = hub.pull("homanp/question-answer-pair")
prompt = hub.pull("poem/task_poem_generater")

chat = prompt |llm|output_parser
# result = chat.invoke({"number_of_pairs": "3","data_format":"Y-m-d","context":"question about school"})
result = chat.invoke({"topic":"Write a poem about cats, in 8 lines"})
print(result)

D:\codes\ai\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


I am a quiz generator assistant, expert in crafting quizzes. Please ask me a quiz-related query.


---
---
##  5. Using PipelinePromptTemplate
- Allows combining multiple prompt templates in sequence.

In [78]:
from langchain.prompts import PromptTemplate, PipelinePromptTemplate

question_one = PromptTemplate.from_template("Translate this text to French: {text}")
question_final = PromptTemplate.from_template("Summarize this in 10 words: {translated_text}")

pipeline_prompt = PipelinePromptTemplate(
    final_prompt=question_final,
    pipeline_prompts = [
         ("translated_text", question_one),
    ]
    
)
final_prompt= pipeline_prompt.format(text="Hello, how are you?")
chain = prompt | llm | output_parser
result = chain.invoke(final_prompt)
print(result)


Here is a summary in 10 words: "Demande de traduction et de résumé en 10 mots."


#### Another Example of Multiple PipelinePromptTemplate
- This code works with 3 templates uncommend the extra templates to add another level of promptTemplate

In [83]:
from langchain.prompts import PromptTemplate, PipelinePromptTemplate

# Step 1: Turn topic into a question
question_template = PromptTemplate.from_template("Turn the following topic into a question: {topic}")
# Step 2: Create an outline for that question
outline_template = PromptTemplate.from_template("Create an outline to answer this question: {question}")
# Step 3: Summarize the outline
summary_template = PromptTemplate.from_template("Summarize this outline in 3 sentences: {outline}")
emoji_template = PromptTemplate.from_template("add Emojis to this summary: {summary}")


pipeline_prompt = PipelinePromptTemplate(
    final_prompt=summary_template,
    # final_prompt=emoji_template,
    pipeline_prompts=[
        ("question", question_template),   # question depends on 'topic'
        ("outline", outline_template),     # outline depends on 'question'
        # ("summary", summary_template)    # summary depends on 'outline'
    ]
)

chain = pipeline_prompt | llm | output_parser
result = chain.invoke({"topic":"Artificial Intelligence"})
print(result)


## see the prompts all together
merged_prompt = pipeline_prompt.format(topic="Artificial Intelligence")

Here is a summary of the outline in 3 sentences:

The question to be answered is "What is the significance of artificial intelligence?" To address this question, the outline explores the definition and history of artificial intelligence, its current applications and benefits, and its potential future implications on society and humanity. Through this examination, the outline aims to provide a comprehensive understanding of the significance of artificial intelligence in today's world.


#### if you combine the pipeline prompt using .format, it merges the question together
- If you pass the merged text from pipeline_prompt.format into a PromptTemplate, the result will not be the same
- in a way, PipelinePrompt act as if you actually did multiple calls to the llm

In [84]:
 merged_prompt

'Summarize this outline in 3 sentences: Create an outline to answer this question: Turn the following topic into a question: Artificial Intelligence'

In [86]:
prompt_text= "Artificial Intelligence.Turn the following topic into a question.Create an outline to answer this question.Summarize this outline in 3 sentences"
prompter = PromptTemplate.from_template(prompt_text)
chain = prompter | llm | output_parser
result = chain.invoke({"topic":"Artificial Intelligence"})
print(result)

Please provide the topic, and I'll convert it into a question, create an outline to answer it, and summarize the outline in 3 sentences.

(Note: If you don't provide a topic, I'll assume you want to use "Artificial Intelligence" as the topic.)

**Topic:** Artificial Intelligence

**Question:** What are the current applications and potential future developments of Artificial Intelligence?

**Outline:**

I. Introduction

* Brief overview of AI and its growing importance
* Thesis statement: AI has numerous current applications and potential future developments that are transforming various industries.

II. Current Applications of AI

* Natural Language Processing (NLP) in virtual assistants and chatbots
* Machine Learning in image and speech recognition
* AI in healthcare for diagnosis and treatment
* AI-powered robots in manufacturing and logistics
* AI in finance for risk management and investment analysis

III. Potential Future Developments of AI

* Advancements in Deep Learning and Ne

---
---
## 6. Using StringPromptTemplate (Custom)
- You can subclass StringPromptTemplate to define a custom way to format prompts.

In [97]:
from langchain.prompts import StringPromptTemplate

class CustomPrompter(StringPromptTemplate):
    def format(self, **kwargs):
        return f"Write a 3 line poem on {kwargs['input']}"


prompt = CustomPrompter(input_variables=["input"])
new_prompt=prompt.format(input="Langchain")

new_prompt
        

'Write a 3 line poem on Langchain'